# These worked on Databricks so it should work when you check them

In [ ]:
from pyspark.sql import functions as F
import plotly.express as px
import matplotlib.pyplot as plt




In [ ]:
cbg = spark.table("safegraph.censusblockgroup_pkmap")
patterns = spark.table("safegraph.patterns")
places = spark.table("safegraph.places")
tract_members = spark.table("membership.tract_membership")
tract_member_info = spark.table("membership.tract_membership_populations")
census_table = spark.table("safegraph.censusmapping")
tract_spatial = spark.table("census.tract_spatial")
tract_distance_temple = spark.table("census.tract_distance_to_nearest_temple")

In [ ]:
census_block = spark.table("safegraph.censusblock_table")
tract_population = spark.table("census.tractpopulations")


# Simple Feature: Population Density

In [ ]:
joined_data = census_block.join(tract_population, census_block.tractcode == tract_population.tractcode, how="inner")


In [ ]:
display(joined_data)

In [ ]:
density_data = joined_data.withColumn("population_density", F.col("population") / F.col("land_m"))


In [ ]:
tract_members = tract_members.withColumnRenamed("home", "tractcode")
display(tract_members)

In [ ]:
display(density_data)

In [ ]:
visualization_data = density_data.join(tract_members, on="tractcode", how="inner")

display(visualization_data)



In [ ]:
display(visualization_data)

In [ ]:
visualization_data = visualization_data.select("lat", "long", "population_density", tract_members.tractcode, "active_members_estimate")

display(visualization_data)


In [ ]:
pandas_density_data  = visualization_data.select("lat", "long", "population_density").limit(100000).toPandas()



In [ ]:
pandas_density_data = pandas_density_data.dropna(subset=["population_density"])

In [ ]:
fig = px.scatter_mapbox(pandas_density_data, 
                        lat="lat", 
                        lon="long", 
                        size="population_density", 
                        color="population_density",
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=50, 
                        zoom=10)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()



# Actual Feature

## Had trouble pulling it down but this is it if you actually want to have it

In [ ]:
actual_feature = visualization_data.select("population_density", tract_members.tractcode)

In [ ]:
display(visualization_data.select("population_density", "active_members_estimate"))


# Complex Feature: Average Visit Duration by Tract

In [ ]:
patterns = spark.table("safegraph.patterns")
places = spark.table("safegraph.places")
census_mapping = spark.table("safegraph.censusmapping")
tract_members = spark.table("membership.tract_membership")



In [ ]:
lds_churches = places.filter(
    (F.col("top_category") == "Religious Organizations") &
    (F.col("location_name").rlike("Latter|latter|Saints|saints|LDS|\b[Ww]ard\b")) &
    (F.col("location_name").rlike("^((?!Reorganized).)*$")) &
    (F.col("location_name").rlike("^((?!All Saints).)*$")) &
    (F.col("location_name").rlike("^((?![cC]ath).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]ody).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]eter).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]atholic).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]res).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]inist).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]ission).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]orship).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]ain).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]aptist).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]eth).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]vang).)*$")) &
    (F.col("location_name").rlike("^((?![Ll]utheran).)*$")) &
    (F.col("location_name").rlike("^((?![Oo]rthodox).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]piscopal).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]abernacle).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]arvest).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ssem).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]edia).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]artha).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]hristian).)*$")) &
    (F.col("location_name").rlike("^((?![Uu]nited).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]ellowship).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]esl).)*$")) &
    (F.col("location_name").rlike("^((?![C]cosmas).)*$")) &
    (F.col("location_name").rlike("^((?![Gg]reater).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]rison).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]ommuni).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]lement).)*$")) &
    (F.col("location_name").rlike("^((?![Vv]iridian).)*$")) &
    (F.col("location_name").rlike("^((?![Dd]iocese).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]istory).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]chool).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]hougt).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]oliness).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]artyr).)*$")) &
    (F.col("location_name").rlike("^((?![Jj]ames).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]ellowship).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]ouse).)*$")) &
    (F.col("location_name").rlike("^((?![Gg]lory).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]nglican).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]oetic).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]anctuary).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]quipping).)*$")) &
    (F.col("location_name").rlike("^((?![Jj]ohn).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ndrew).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]manuel).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]edeemed).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]erfecting).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]ngel).)*$")) &
    (F.col("location_name").rlike("^((?![Aa]rchangel).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]icheal).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]hought).)*$")) &
    (F.col("location_name").rlike("^((?![Pp]ariosse).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]osmas).)*$")) &
    (F.col("location_name").rlike("^((?![Dd]eliverance).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]ociete).)*$")) &
    (F.col("location_name").rlike("^((?![Tt]emple).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]eminary).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]mployment).)*$")) &
    (F.col("location_name").rlike("^((?![Ii]nstitute).)*$")) &
    (F.col("location_name").rlike("^((?![Cc]amp).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]tudent).)*$")) &
    (F.col("location_name").rlike("^((?![Ee]ducation).)*$")) &
    (F.col("location_name").rlike("^((?![Ss]ocial).)*$")) &
    (F.col("location_name").rlike("^((?![Ww]welfare).)*$")) &
    (F.col("location_name").rlike("^((?![Cc][Ee][Ss]).)*$")) &
    (F.col("location_name").rlike("^((?![Ff]amily).)*$")) &
    (F.col("location_name").rlike("^((?![Mm]ary).)*$")) &
    (F.col("location_name").rlike("^((?![Rr]ussian).)*$")) &
    (F.col("location_name").rlike("^((?![Bb]eautif).)*$")) &
    (F.col("location_name").rlike("^((?![Hh]eaven).)*$")) &    
    (F.col("location_name").rlike("^((?!Inc).)*$")) &
    (F.col("location_name").rlike("^((?!God).)*$")))\
    .dropDuplicates(["placekey"])

In [ ]:
patterns_lds_joined = patterns.join(lds_churches, on="placekey", how="inner")


In [ ]:
pattern_lds_tract_joined = patterns_lds_joined.join(census_mapping, on="placekey", how="inner")


In [ ]:
average_visit_duration_by_tract_lds = pattern_lds_tract_joined.groupBy("tract").agg(F.avg("median_dwell").alias("average_visit_duration"))


In [ ]:
final_data_lds = average_visit_duration_by_tract_lds.join(tract_members, average_visit_duration_by_tract_lds.tract == tract_members.home)


In [ ]:
sampled_final_data_lds = final_data_lds.sample(fraction=0.1).limit(1500).toPandas()


plt.figure(figsize=(10, 6))
plt.scatter(sampled_final_data_lds["average_visit_duration"], sampled_final_data_lds["active_members_estimate"])
plt.title("Average Visit Duration at LDS Churches vs Active Members Estimate")
plt.xlabel("Average Visit Duration")
plt.ylabel("Active Members Estimate")
plt.grid(True)
plt.show()


In [ ]:
fig = px.scatter(sampled_final_data_lds, x="average_visit_duration", y="active_members_estimate", 
                 title="Average Visit Duration at LDS Churches vs Active Members Estimate")
fig.show()